# Hola &#x1F600;

Soy **Hesus Garcia**, revisor de código de Triple Ten, y voy a examinar el proyecto que has desarrollado recientemente. Si encuentro algún error, te lo señalaré para que lo corrijas, ya que mi objetivo es ayudarte a prepararte para un ambiente de trabajo real, donde el líder de tu equipo actuaría de la misma manera. Si no puedes solucionar el problema, te proporcionaré más información en la próxima oportunidad. Cuando encuentres un comentario,  **por favor, no los muevas, no los modifiques ni los borres**. 

Revisaré cuidadosamente todas las implementaciones que has realizado para cumplir con los requisitos y te proporcionaré mis comentarios de la siguiente manera:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si todo está perfecto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>

Puedes responderme de esta forma:
<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class=“tocSkip”></a>
</div>

</br>

**¡Empecemos!**  &#x1F680;


# Introducción

Se busca el calculo de la eficiencia de una minera de extracción de oro por Zyfra

## Inicialización

### Librerías a usar

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import (make_scorer, mean_absolute_error)
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler

from collections import defaultdict


### Carga de datasets

In [ ]:
# Carga de datasets a trabajar
try:
    train = pd.read_csv('gold_recovery_train.csv')
    test = pd.read_csv('gold_recovery_test.csv')
    full = pd.read_csv('gold_recovery_full.csv')
except:
    train = pd.read_csv('/datasets/gold_recovery_train.csv')
    test = pd.read_csv('/datasets/gold_recovery_test.csv')
    full = pd.read_csv('/datasets/gold_recovery_full.csv')

In [ ]:
train.info()
train.head()

In [ ]:
test.info()
test.head()

In [ ]:
full.info()
full.head()

Revisando los datos de cada uno de los conjuntos de datos se trabajará lo siguiente:
    1.- Columna `data` se transformará a DateTime en los tres datasets.
    2.- Verificar los valores ausentes, al no tener los datos completos se eliminarán las filas con valores ausentes.

### Preprocesamiento

Se trabajará la columna `data` para transforma a tipo DateTime

In [ ]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
full['date'] = pd.to_datetime(full['date'])

Se eliminan los valores ausentes en los datasets

In [ ]:
train = train.dropna()
test = test.dropna()
full = full.dropna()

Se buscará usar las mismas columnas en cada dataset para realizar el análisis de mejor forma.

In [ ]:
list_test = list(test.columns.values)+ ['rougher.output.recovery','final.output.recovery']
train_df = train.loc[:,list_test]
train_df.info()

Se crean las columnas de target a usar para entrenar los modelos

In [ ]:
target_columns = full[['date','rougher.output.recovery','final.output.recovery']]
test_df = test.merge(target_columns, how='left', on='date')
test_df.info()

### Calculo de recuperados

Para verificar que el cálculo de recuperados es correcto se realizará un cálculo con las siguientes columnas:

    - rougher.input.infeed_au La cual es la concentración inicial de oro alimentado.
    - rougher.output.concentrate_au Concentración de oro antes de la flotación.
    - rougher.output.tail_au Concentración de oro después de la flotación.

Con estos datos se va a calcular el porcentaje de oro para cada fila del data set train.

In [ ]:
#Función para calculo de procentaje de oro

def calculo_concentrado(ini_au, conc_au, tail_au):
    recuperado = (conc_au*(ini_au-tail_au))/(ini_au*(conc_au-tail_au))*100
    print(recuperado.head())
    return recuperado

In [ ]:
#Valores presentados en dataset
recovery = train['rougher.output.recovery']

In [ ]:
concentrado = train['rougher.output.concentrate_au']
alimentacion = train['rougher.input.feed_au']
colas = train['rougher.output.tail_au']

recuperado_calculado = calculo_concentrado(alimentacion,concentrado,colas)

Se calcula el Error absoluto medio de recuperado_calculado contra recovery

In [ ]:
mae = mean_absolute_error(recuperado_calculado, recovery)
mae

El error es tan pequeño que nuestro cálculo de porcentaje de recuperación de oro es correcto, por lo que procederemos a lo siguiente.

### Análisis set de prueba

El dataset de prueba, no contiene las columnas referentes a la concentración de plata, plomo, solvente, oro finales.
Así mismo tampoco tiene los datos finales de las colas de plata, plomo, solvente y oro.
Para tener todas las características las columnas floatbank10_sulfate_to_au_feed, floatbank11_sulfate_to_au_feed, and au_pb_ratio también están ausentes.

## Análisis de datos

### Cálculo de concentraciones por metal

Se definirá una función para mostrar los cambios de concentración de los diferentes metales (oro, plata, plomo) en las diferentes etapas del proceso.

In [ ]:
def metal_conc(data, etapas):
    for etapa in etapas:
        data[etapa].hist(legend=True, figsize=(9,6),alpha=0.65)
        plt.xlabel("Concentración")

In [ ]:
#Se definen las concentraciones de los metales en las diferentes etapas:

etapa_oro = ['rougher.output.concentrate_au',
             'primary_cleaner.output.concentrate_au',
             'secondary_cleaner.output.tail_au',
             'final.output.concentrate_au']

etapa_plata = ['rougher.output.concentrate_ag',
               'primary_cleaner.output.concentrate_ag',
               'secondary_cleaner.output.tail_ag',
               'final.output.concentrate_ag']

etapa_plomo = ['rougher.output.concentrate_pb',
               'primary_cleaner.output.concentrate_pb',
               'secondary_cleaner.output.tail_pb',
               'final.output.concentrate_pb']

#### Oro

In [ ]:
metal_conc(train,etapa_oro)

#### Plata 

In [ ]:
metal_conc(train,etapa_plata)

#### Plomo

In [ ]:
metal_conc(train,etapa_plomo)

Como se muestra en los histrogramas, conforme avanza el proceso se espera tener una mayor concentración de oro, mientras que la concentración de plata y plomo disminuyen.

### Tamaño de partículas

Se va a comparar el tamaño de partícula en los dataset de entrenamiento *(train)* y de prueba *(test)*.

In [ ]:
train['rougher.input.feed_size'].hist(bins=50, 
                                      alpha=0.5, 
                                      label="Tamaño de partícula entrenamiento")

test['rougher.input.feed_size'].hist(bins=50, 
                                     alpha=0.5, 
                                     label="Tamaño de partícula prueba")

plt.legend()
plt.xlabel("Tamaño de partícula")
plt.ylabel("Frecuencia")
plt.show()

El histograma no muestra gran diferencia en la frecuencia de los tamaños de partículas por lo que los cálculos de estimaciones serán consistentes.

###  Concentración de mezcla a diferentes etapas

Se graficarán la sumas de las concentraciones de los metales en diferentes etapas, esto con el fin de determinar los valores fuera de rango que causarían ruido en nuestro modelo de predicción.

In [ ]:
#Usando del dataset full realizaremos los histogramas

full['total_mixture'] = full[['rougher.input.feed_au',
                              'rougher.input.feed_ag',
                              'rougher.input.feed_pb']].sum(axis=1, skipna=True)
full['total_mixture'].hist(bins=30, figsize=(9,6))
plt.show()

In [ ]:
full['total_roughermix'] = full[['rougher.output.concentrate_au',
                                 'rougher.output.concentrate_ag',
                                 'rougher.output.concentrate_pb']].sum(axis=1,skipna=True)
full['total_roughermix'].hist(bins=30, figsize=(9,6))
plt.show()

In [ ]:
full['total_finalmix'] = full[['final.output.concentrate_au',
                                 'final.output.concentrate_ag',
                                 'final.output.concentrate_pb']].sum(axis=1,skipna=True)
full['total_finalmix'].hist(bins=30, figsize=(9,6))
plt.show()

Los histrogramas nos muestran muchos valores fuera de rango, por lo que los removeremos para no causar ruido en nuestros calculos. También se muestra que los datos están cargados hacia la derecha.

In [ ]:
#Para buscar los valores fuera de rangos u outliers, se realizará un cálculo de los valores intercuartiles

def outliers_drop(data, columnas):
    mapeo = defaultdict(list)
    for columna in columnas:
        q1 = data[columna].quantile(0.25)
        q3 = data[columna].quantile(0.75)
        iqr = q3 - q1
        minimo = q1-1.5*iqr
        maximo = q3+1.5*iqr
        mapeo[columna] = (minimo, maximo)
    df = data
    for a, b in mapeo.items():
        df = df.loc[(df[a] > b[0]) & (df[a] < b[1])]
        
    return df

In [ ]:
parametros = ['total_mixture','total_roughermix','total_finalmix']

print(full.shape)

full_fin = outliers_drop(full, parametros)

print(full_fin.shape)

Se muestran los histogramas nuevamente

In [ ]:
full_fin['total_mixture'].hist(bins=30)
plt.show()

In [ ]:
full_fin['total_roughermix'].hist(bins=30)
plt.show()

In [ ]:
full_fin['total_finalmix'].hist(bins=30)
plt.show()

Ya con los datos más parecidos a una distribución normal, procederemos a entrenar y probar nuestro modelo.

In [ ]:
#Se agrega la columna total_mixture al dataset de entrenamiento
columnas = full_fin[['date','total_mixture']]
train_final = train_df.merge(columnas, how='left', on='date')
train_final.shape[0]

In [ ]:
#Se quitan las filas con valores nulos en total_mixture
train_final = train_final[train_final.total_mixture.notnull()]
train_final = train_final.drop(['total_mixture'], axis=1)
train_final.shape[0]


Se hará lo mismo con el dataset de prueba

In [ ]:
test_final = test_df.merge(columnas, how='left', on='date')
test_final.shape[0]

In [ ]:
test_final = test_final[test_final.total_mixture.notnull()]
test_final = test_final.drop(['total_mixture'], axis=1)
test_final.shape[0]

Con estos datasets trabajaremos los modelos

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
¡Muy bien! 👏👏 Los cálculos de esta sección están correctos y eso es un gran logro. Sigue así y verás cómo poco a poco te irás convirtiendo en un experto en esta área. 💪💻</div>

## Modelado y Resultados

### Cálculo de sMAPE

In [ ]:
def smape_cal(target, prediction):
    
    def smape(target, prediction):
        smape = np.mean((np.abs(target-prediction)/((np.abs(target)+np.abs(prediction))/2)))*100
        return smape
    
    smape_r = smape(target[:,0],prediction[:,0])
    smape_f = smape(target[:,1],prediction[:,1])
    smape_final = 0.25*smape_r + 0.75*smape_f
    return smape_final

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
La función está correctamente definida. 
</div>

### Entrenamiento de modelos

#### Características y resultados

In [ ]:
targets = ['rougher.output.recovery','final.output.recovery']

train_features = train_final.drop(targets, axis=1)
train_target = train_final[targets]

test_features = test_final.drop(targets,axis=1)
test_target = test_final[targets]

In [ ]:
smape_scorer = make_scorer(smape_cal)

In [ ]:
scaler = StandardScaler()
train_features = train_features.set_index('date')
test_features = test_features.set_index('date')

features = list(train_features.columns)
scaler.fit(train_features[features])

train_features[features] = scaler.transform(train_features[features])
test_features[features] = scaler.transform(test_features[features])

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Perfecto! Preveniste el data leakage. El escalado de datos se realiza por separado para los conjuntos de entrenamiento y prueba.
Cuando escalas tus datos, estás ajustando sus valores para que tengan una escala uniforme.
Imagina esto: Si escalas todo el conjunto de datos (entrenamiento + prueba) juntos, es posible que los valores de prueba influyan en la escala de entrenamiento y viceversa.
    


</div>

#### RandomForestRegressor

In [ ]:
rfr_model = RandomForestRegressor(random_state=1234, n_estimators=15)

rfr_train_smape = cross_validate(rfr_model, train_features, train_target.to_numpy(), scoring=smape_scorer, cv=5)

for a,b in rfr_train_smape.items():
    print(a,b)

score_final = rfr_train_smape['test_score']
print("sMAPE promedio: ", np.sum(score_final)/5)

#### LinearRegression

In [ ]:
lr_model = LinearRegression()

lr_train_smape = cross_validate(lr_model, 
                                train_features, 
                                train_target.to_numpy(), 
                                scoring=smape_scorer, 
                                cv=5)

for a,b in lr_train_smape.items():
    print(a,b)

score_final = lr_train_smape['test_score']
print("sMAPE promedio: ", np.sum(score_final)/5)

#### DecisionTreeRegressor

In [ ]:
for depth in range(1,15):
    dcr_model = DecisionTreeRegressor(random_state=1234, max_depth=depth)
    
    dcr_train_smape = cross_validate(dcr_model, 
                                     train_features, 
                                     train_target.to_numpy(), 
                                     scoring=smape_scorer, 
                                     cv=5)
    
    for a,b in dcr_train_smape.items():
        print(a,b)

    score_final = dcr_train_smape['test_score']
    print("Depth: ", depth)
    print("sMAPE promedio: ", np.sum(score_final)/5)

El modelo DecisionTreeRegresor es el que nos arroja menor sMAPE, con una profundidad máxima de 1, por lo que utilizaremos estos valores para entrenar y probar el modelo.

<div class="alert alert-block alert-success">
<b>Comentario del Revisor</b> <a class="tocSkip"></a>


He revisado tu implementación de los modelos RandomForestRegressor, LinearRegression y DecisionTreeRegressor, y quisiera destacar varios aspectos positivos de tu enfoque.

1. <b>Selección de Modelos y Parámetros:</b> Has hecho un excelente trabajo al seleccionar una variedad de modelos  de regressón. para probar en tu conjunto de datos

2. <b>Uso de Cross-Validation:</b> Tu implementación de validación cruzada fue adecuada. 

3. <b>Implementación del sMAPE Custom Scorer:</b> La utilización del sMAPE (Symmetric Mean Absolute Percentage Error) como métrica es adecuada para problemas de regresión, especialmente en contextos donde los errores relativos son más importantes. Además, la creación de un 'scorer' personalizado para esta métrica demuestra un nivel avanzado de habilidad en la manipulación de datos y evaluación de modelos.

4. <b>Análisis y Elección del Modelo Final:</b> La decisión de elegir el modelo con el menor sMAPE promedio es lógica y bien fundamentada. Tu análisis para llegar a esta conclusión está bien articulado y muestra un razonamiento claro.


### Resultados

In [ ]:
dtr_model = DecisionTreeRegressor(random_state=1234, max_depth=1)

dtr_model.fit(train_features, train_target)

predict = dtr_model.predict(test_features)

smape = smape_cal(test_target.to_numpy(), predict)
print("sMAPE: ", smape)

Nuestro modelo nos un valor sMAPE de 6.589, menor que lo presenta la validación cruzada, con esto damos un valor de certidumbre aceptable a nuestro modelo.

<div class="alert alert-block alert-success">    
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
¡Qué gran trabajo has hecho!  &#128077;  Podemos aprobar el proyecto. <br>
Has demostrado un excelente conocimiento en la construcción de modelos, al eliminar variables innecesarias y procesar  los datos antes de entrenar los modelos. <br>
<br>Quiero felicitarte por un trabajo excepcional y por la calidad de tu análisis. Te animo a que sigas aprendiendo y desafiando tu potencial en los próximos sprints. Estoy seguro de que tus habilidades y conocimientos serán valiosos en el futuro y te permitirán abordar problemas cada vez más complejos con éxito.
</div>